# **TP4 - Grupo 4**
Pedro Paulo Costa Pereira - A88062

Tiago André Oliveira Leite - A91693

In [1]:
from z3 import *

# **Problema - Correção de Programas**

Considere o seguinte programa, em Python anotado, para multiplicação de dois inteiros de precisão limitada a 16 bits.  

```python
assume m >= 0 and n >= 0 and r == 0 and x == m and y == n 

0: while y > 0:
1:    if y & 1 == 1: 
2:            y , r  = y-1 , r+x
3:    x , y = x<<1  ,  y>>1
4: assert r == m * n

```

#### Variaveis Globais

In [2]:
Bits = 16
Limite = 8190

#### Função que declara as variaveis de cada estado

As variáveis são guardadas guardadas num dicionario $s$.
Sendo que representam:
- $s['m']$ ⟶ um bitvector com o valor inicial de x;<br>
- $s['n']$ ⟶ um bitvector com o valor inicial de y;<br>
- $s['x']$ ⟶ um bitvector com o valor de x;<br>
- $s['y']$ ⟶ um bitvector com o valor de y;<br>
- $s['r']$ ⟶ um bitvector com o valor de r;<br>
- $s['pc']$ ⟶ um bitvector com o valor do program counter;<br>

In [172]:
def declare(i):
    s = {}
    s['m'] = BitVec('m'+str(i),Bits)
    s['n'] = BitVec('n'+str(i), Bits)
    s['x'] = BitVec('x'+str(i), Bits)
    s['y'] = BitVec('y'+str(i), Bits)
    s['r'] = BitVec('r'+str(i), Bits)
    s['pc'] = BitVec('pc'+str(i), Bits)
    
    return s

#### Função que adiciona as restrições do estado inicial

As restrições no estado inicial são:
- $m \ge 0 \quad \land\ \quad n \ge 0 \quad \land\ \quad r = 0 \quad \land\ \quad x = m \quad \land\ \quad y = n \quad \land\ \quad pc = 0 $

In [4]:
def init(s):
    return And(s['m'] >= 0,s['m']<Limite, s['n'] >= 0,s['n']<Limite, s['r'] == 0, s['x'] == s['m'], s['y'] == s['n'], s['pc'] == 0) 

#### Função que adiciona as restrições associadas a cada transição

In [111]:
def trans(s,p):
    t01 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y']>0,s['pc']==0, p['pc']==1, s['r'] == p['r'])
    
    t12 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y'] & 1 == 1,s['pc']==1, p['pc']==2, s['r'] == p['r'])
    
    t23 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], p['y']==s['y']-1,
              s['pc']==2, p['pc']==3, p['r'] == s['r']+s['x'])
    
    t13 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y'] & 1 != 1,s['pc']==1, p['pc']==3, s['r'] == p['r'])
    
    t30 = And(s['m']==p['m'], s['n']==p['n'], p['x']==s['x']<<1, p['y']==s['y']>>1,
              s['pc']==3, p['pc']==0, s['r'] == p['r'])
    
    t04 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['y']<=0,s['pc']==0, p['pc']==4, s['r'] == p['r'])
    
    t44 = And(s['m']==p['m'], s['n']==p['n'], s['x']==p['x'], s['y']==p['y'],
              s['pc']==4, p['pc']==4, s['r'] == p['r'])
    
    return Or(t01, t12, t23, t13, t30, t04, t44)

#### Função que gera um traço de execução do sistema com k estados

A função vai simular, atravês do solver Z3, uma execução do sistema com **k** estados, $[E_0 .. E_{k-1}]$, e **k-1** transições, com as restrições codificadas anteriormente. Sendo $E_0$ o estado inicial. 

Caso o sistema seja satisfazivel, vão ser imprimidas, estado a estado, os valores das variáveis do sistema.

In [6]:
def gera_traco(declare,init,trans, k):
    s = Solver()
    traco = {}
    for i in range(k):
        traco[i] = declare(i)
    s.add(init(traco[0]))
    for i in range(k-1):
        s.add(trans(traco[i],traco[i+1]))
    status = s.check()
    if status == sat:
        m = s.model()
        for i in range(k):
            for v in traco[i]:
                if traco[i][v].sort() == RealSort():
                    print(v,'=', float(m[traco[i][v]].numerator_as_long())/float(m[traco[i][v]].denominator_as_long()))
                else:
                    print(v,"=",m[traco[i][v]])
            print('')
    elif status == unsat:
        print("Não ha execuções possiveis")
    else:
        print("Resultado impossivel de obter!")

In [7]:
gera_traco(declare,init,trans, 20)

m = 4095
n = 2560
x = 4095
y = 2560
r = 0
pc = 0

m = 4095
n = 2560
x = 4095
y = 2560
r = 0
pc = 1

m = 4095
n = 2560
x = 4095
y = 2560
r = 0
pc = 3

m = 4095
n = 2560
x = 8190
y = 1280
r = 0
pc = 0

m = 4095
n = 2560
x = 8190
y = 1280
r = 0
pc = 1

m = 4095
n = 2560
x = 8190
y = 1280
r = 0
pc = 3

m = 4095
n = 2560
x = 16380
y = 640
r = 0
pc = 0

m = 4095
n = 2560
x = 16380
y = 640
r = 0
pc = 1

m = 4095
n = 2560
x = 16380
y = 640
r = 0
pc = 3

m = 4095
n = 2560
x = 32760
y = 320
r = 0
pc = 0

m = 4095
n = 2560
x = 32760
y = 320
r = 0
pc = 1

m = 4095
n = 2560
x = 32760
y = 320
r = 0
pc = 3

m = 4095
n = 2560
x = 65520
y = 160
r = 0
pc = 0

m = 4095
n = 2560
x = 65520
y = 160
r = 0
pc = 1

m = 4095
n = 2560
x = 65520
y = 160
r = 0
pc = 3

m = 4095
n = 2560
x = 65504
y = 80
r = 0
pc = 0

m = 4095
n = 2560
x = 65504
y = 80
r = 0
pc = 1

m = 4095
n = 2560
x = 65504
y = 80
r = 0
pc = 3

m = 4095
n = 2560
x = 65472
y = 40
r = 0
pc = 0

m = 4095
n = 2560
x = 65472
y = 40
r = 0
pc = 1



## 1. Provar por indução a terminação deste programa

Para provar que o programa termina temos que encontrar um variante, que respeite as seguintes propriedades:
- O variante nunca é negativo, ou seja, $G\ (V(s) \ge 0)$
- O variante descresce sempre (estritamente) ou atinge o valor 0, ou seja, $G\ (\forall s' . \mathit{trans}(s,s') \rightarrow (V(s') < V(s) \vee V(s') = 0))$
- Quando o variante é 0 verifica-se necessariamente $pc = 4$, ou seja, $G\ (V(s)=0 \rightarrow s['pc'] = 4)$.


#### Função que prova uma propriedade atraves por indução

In [171]:
def induction_always(declare,init,trans,prop):
    s = Solver()
    state = declare(0)
    s.add(init(state))
    s.add(Not(prop(state)))
    status = s.check()
    assert(status != unknown)
    if (status == sat):
        print("Não é verdade no estado inicial")
        m = s.model()
        for v in state:
            print(v, '=', m[state[v]])
        return
    
    s = Solver()
    pre = declare(0)
    pos = declare(1)
    s.add(init(pre))
    s.add(prop(pre))
    s.add(trans(pre,pos))
    s.add(Not(prop(pos)))
    status = s.check()
    assert(status != unknown)
    if (status == sat):
        print("Não é possivel provar o passo indutivo no estado")
        m = s.model()
        print('pre')
        for v in pre:
            print(v, '=', m[pre[v]])
        print('pos')
        for v in pos:
            print(v, '=', m[pre[v]])
        return
    print("A propriedade é valida!")

### Propriedades do variante

#### Variante

In [9]:
def variante(state):
    return 4*state['y']-state['pc']+4

#### Nâo negativo

In [10]:
def naonegativo(state):
    return variante(state) >=0

In [11]:
induction_always(declare,init,trans, naonegativo)

A propriedade é valida!


#### Descrescente

In [12]:
def decrescente(state):
    proximo = declare(-1)
    decresce = variante(proximo) < variante(state)
    zero = variante(proximo) == 0
    formula = Implies(trans(state,proximo),Or(zero,decresce))
    return (ForAll(list(proximo.values()),formula))

In [13]:
induction_always(declare,init,trans,decrescente)

A propriedade é valida!


#### Util

In [14]:
def utilidade(state):
    return (Implies(variante(state) == 0, state['pc'] == 4))

In [15]:
induction_always(declare,init,trans,utilidade)

A propriedade é valida!


## 2. Correção total

### a. Forma recursiva do programa codificado em LPA (linguagem de programas anotados).

$P \equiv$ assume  $\phi$ ;  $W$ ; assert $\varphi$ ; skip <br>

$\phi \equiv  m \ge 0 \ \ \land\ \ \ n \ge 0  \ \ \land\ \ \ x = m \ \ \land\ \ \ y = n \ \ \land\ \ \ r = 0$ <br>

$\varphi \equiv  r = m \ \ x  \ \ n$ <br>

$W \equiv$ assume $b$;  $S$;  $W$ || assume $\neg b$;<br>

$b \equiv y > 0$ <br>

$S \equiv$ {assume( $y \ \& \ 1 = 1)$; $y \leftarrow y-1$; $r \leftarrow r + x$ || skip}; $C$<br>

$C \equiv x \leftarrow x << 1$ ; $y \leftarrow y >> 1$ <br>


### b. Invariante mais fraco que assegura a correção do programa.

Para provar a correção do programa temos que encontrar um invariante de ciclo, $\theta$, que respeite as seguintes propriedades:
- Inicialização, ou seja, $\phi \rightarrow \theta$;
- Preservação, ou seja, $W \land\ \theta  \rightarrow \theta$;
- Utilidade, ou seja, $\neg b \land\ \theta \rightarrow \varphi$;

Um cadidato a invariante de ciclo pode ser:

$\theta \equiv \ m \ \times \ n \ = \ r \ + \ x \ \times \ y \ \land\ y \ge 0 \ \land\ n \ge y \land\ n \ge 0 \land\ m \ge 0$


#### Variaveis

In [141]:
m, n, r, x, y = BitVecs('m n r x y',Bits)

#### Invariante

In [168]:
invariante = And(x*y+r==m*n,y>=0, n>=y,m>=0,n>=0)

#### Função que prova as propriedades

In [148]:
def prove(f):
    s = Solver()
    s.add(Not(f))
    c = s.check()
    if c == unsat:
        print("válido")
    elif c == sat:
        print("inválido")
        print(s.model())
    else:
        print("desconhecido")

#### Inicialização

In [156]:
prove(Implies(And(m>=0,n>=0,r==0,x==m,y==n),invariante))

válido


#### Preservação

In [170]:
#pre = And(y>0,invariante)
pre = And(y>0,n<=20,m<=20,invariante)
vcthen = Implies(y & 1 == 1, substitute (substitute(substitute(substitute(invariante,(y,y>>1)),(x,x<<1)),(r,r+x)),(y,y-1)))
vcelse = Implies(y & 1 == 0, substitute (substitute(invariante,(y,y>>1)),(x,x<<1)))
prove(Implies(pre,And(vcthen,vcelse)))

válido


#### Utilidade

In [157]:
prove(Implies(And(y==0,invariante),r==m*n))

válido


### c. Unfold com parametro limite N para provar correção do progrma.

Para provar a correção por unfold com um parametro limite **N**, temos que "esticar" **(unfold)** a execução do ciclo numa aproximação formada por uma sequência finita, numero finito, de execuções do “corpo do ciclo”.

Ora, sendo **N** o parametro limite e uma vez que em cada execução do ciclo existe uma sequencia de no máximo 4 estados, sabemos entao que o ciclo vai executar um número finito de vezes. Portanto, podemos definir um número maximo de $4 \times N + 1$ estados.   

Assim sendo, para provar o invariante, vamos gerar um traço de execução do programa com $4 \times N + 1$ estados,$[S_0 .. S_{k}]$com, onde serão acrescentadas a seguinte restrição:<br>
- $S_0['m'] < N \ \land\ \ S_0['n'] < N $ <br>
- $\neg \theta (S_k)$

Por fim utilizamos o solver do z3 para realizar o traço de execução e caso o **status** seja **unsat** isso significa que o programa está correto.

#### Invariante

In [134]:
def inv(s):
    return And(s['m']*s['n'] == s['x']*s['y'] + s['r'], s['y']>=0, s['n']>=s['y'],s['n']>=0,s['m']>=0)

#### Função que prova a correção do programa por unfold

In [139]:
def unfold(declare,init,trans,prop,N):
    k = N*4 +1
    s = Solver()
    traco = {}
    for i in range(k):
        traco[i] = declare(i)
    s.add(init(traco[0]))
    s.add(And(traco[0]['m']<N,traco[0]['m']<N))
    for i in range(k-1):
        s.add(trans(traco[i],traco[i+1]))
    s.add(Not(prop(traco[k-1])))
    status = s.check()
    if status == sat:
        m = s.model()
        for i in range(k):
            print(i)
            for v in traco[i]:
                print(v, "=", m[trace[i][v]])
        return
    print("O programa está correto")
    

In [140]:
unfold(declare,pre,trans,inv,20)

O programa está correto
